In [2]:
import os
import pandas as pd
import numpy as np
import gzip

os.chdir("../Data")


In [3]:
f = gzip.GzipFile('HestonTrainSet.txt.gz', "r")
dat=np.load(f)
xx=dat[:,:5]
yy=dat[:,5:]
print(xx.shape)
strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])

(12000, 5)


Heston model:

\begin{align}
& dS_t = \mu S_t d_t + \sqrt{v_t}S_tdW_t^S \\ 

& dv_t = \kappa(\theta-v_t)dt+\nu\sqrt{v_t}dW_t^v \\ 

& dW_t^S  dW_t^v = \rho dt
\end{align}

# Step 1: Data Generation


## **Inputs:** 
###  1. $\rho$ spot-vol correlation,
###  2. $\nu$ vol of vol
###  3. $\theta$ long term vol
###  4. $\kappa$ speed of mean reversion 
###  5. $\xi_{0}$ forward variance curve
    
    
## **Outputs:**
### Implied volatility surface (11x8=88 points)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    yy, xx, test_size=0.15, random_state=42)
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
scale2=  StandardScaler()
y_train_transform = scale.fit_transform(y_train)
y_test_transform = scale.transform(y_test)
x_train_transform = scale2.fit_transform(X_train)
x_test_transform = scale2.transform(X_test)

In [5]:
def xtransform(X_train,X_test):
    return [scale2.transform(X_train),scale2.transform(X_test)]

    
[x_train_transform,x_test_transform]=xtransform(X_train,X_test)

def xinversetransform(x):
    return scale2.inverse_transform(x)

In [6]:
ub=[0.04,-0.1,1.0,0.2,10.0]
lb=[0.0001,-0.95,0.01,0.01,1]
def myscale(x):
    res=np.zeros(5)
    for i in range(5):
        res[i]=(x[i] - (ub[i] + lb[i])*0.5) * 2 / (ub[i] - lb[i])
        
    return res
def myinverse(x):
    res=np.zeros(5)
    for i in range(5):
        res[i]=x[i]*(ub[i] - lb[i]) *0.5 + (ub[i] + lb[i])*0.5
        
    return res

In [ ]:
y_train_transform = np.array([myscale(y) for y in y_train])
y_test_transform = np.array([myscale(y) for y in y_test])